In [ ]:
get_ipython().system('pip install yfinance')
get_ipython().system('pip install PyPortfolioOpt')
get_ipython().system('pip install cvxopt')
get_ipython().system('pip install cvxpy')
get_ipython().system('pip install matplotlib')
get_ipython().system('pip install sklearn')
import pypfopt
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation
get_ipython().system('pip install qiskit')
get_ipython().run_line_magic('matplotlib', 'inline')
from qiskit.finance import QiskitFinanceError
from qiskit.finance.data_providers import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
def covariance(prices):
    sample_cov = risk_models.sample_cov(prices, frequency=252)
    plotting.plot_covariance(sample_cov, plot_correlation=True)
    return sample_cov

from pypfopt import expected_returns
def returns(prices):
    mu = expected_returns.capm_return(prices)
    return mu

def min_variance(prices):
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(None, S, weight_bounds=(None, None))
    ef.min_volatility()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return(weights)

def Max_Sharpe_port(prices):
    mu = returns(prices)
    S = covariance(prices)
    ef = EfficientFrontier(mu, S) 
    ef.max_sharpe()
    weights = ef.clean_weights()
    pd.Series(weights).plot.pie(figsize=(10,10));
    return weights

from pypfopt import objective_functions
risk = 0.15
def return_given_risk(prices, risk):
    mu = returns(prices)
    S = risk_models.sample_cov(prices)
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg)  # gamme is the tuning parameter
    ef.efficient_risk(0.15)
    weights = ef.clean_weights()
    return weights


def minimize_risk(prices):
    mu = returns(prices)
    S = covariance(prices)
    ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
    ef.add_objective(objective_functions.L2_reg)
    ef.efficient_return(target_return=0.07, market_neutral=True)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return weights

In [ ]:
# sorted by top AUM

tech = ['QQQ', 'VGT', 'XLK', 'ARKK', 'FDN'] 
healthcare = ['XLV', 'VHT','IBB','ARKG','IHI']
financials = ['XLF', 'VFH', 'KRE', 'KBE', 'FAS']
real_estate = ['VNQ', 'SCHH', 'VNQI', 'IYR', 'REET']
energy = ['XLE', 'ICLN', 'AMLP', 'VDE', 'XOP']
materials = ['GDX', 'XLB', 'GDXJ', 'GUNR', 'VAW']
consumer = ['XLY', 'XLC', 'VCR', 'PEJ', 'FXD']
industrials = ['XLI', 'VIS', 'JETS', 'ITA', 'IYT']
telecom = ['VOX', 'NXTG', 'FCOM', 'IYZ', 'IXP']
consumer_staples = ['XLP', 'VDC', 'IEV', 'FSTA', 'IYK']
utilities = ['XLU', 'VPU', 'IGF', 'NFRA', 'PAVE']

all_industries = ['QQQ', 'XLV', 'XLF', 'VNQ', 'XLE', 'GDX', 'XLY', 'XLI', 'VOX', 'XLP', 'XLU']

In [ ]:
tickers = all_industries
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"].dropna(how="all")

In [ ]:
# Long/Short Min Variance
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
ef = EfficientFrontier(None, S, weight_bounds=(None, None))
ef.min_volatility()
weights = ef.clean_weights()
ef.portfolio_performance(verbose=True);
weights

In [ ]:
# Max Sharpe with Sector Constraints

mu = returns(prices)
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()

ef = EfficientFrontier(mu, S)  # weight_bounds automatically set to (0, 1)
ef.max_sharpe()
weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
weights

In [ ]:
from pypfopt import DiscreteAllocation

latest_prices = prices.iloc[-1]  # prices as of the day you are allocating
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=20000, short_ratio=0.3)
alloc, leftover = da.lp_portfolio()
print(f"Discrete allocation performed with ${leftover:.2f} leftover")
alloc

In [ ]:
data = YahooDataProvider(
                 tickers = ["AEO", "ABBY", "AEP", "AAL"],
                 start=datetime.datetime(2018, 1, 1),
                 end=datetime.datetime(2018, 12, 31))
data.run()

num_assets = 4

mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

q = 0.5                   # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets      # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

In [ ]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))


In [ ]:
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
result = exact_eigensolver.run()

print_result(result)

In [ ]:
backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

In [ ]:
backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa = QAOA(qubitOp, cobyla, 3)

qaoa.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = qaoa.run(quantum_instance)

print_result(result)

In [ ]:
get_ipython().system('pip install alpaca-trade-api')

import alpaca_trade_api as tradeapi
# from alpaca_trade_api.rest import TimeFrame
# import pandas as pd

APCA_API_KEY_ID = ''
APCA_API_SECRET_KEY = ''
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

# Setting up API and account details
api = tradeapi.REST(
    APCA_API_KEY_ID,
    APCA_API_SECRET_KEY,
    base_url=APCA_API_BASE_URL,
    api_version='v2')

def market_order_from_list(orderList):
    """
    Creates live market buy orders which expire at end of the day from a list of tuples in the form:
    orderList = [('AAPL', '10'), ('JPM', '5')]
    """
    for ticker, amount in orderList:
        api.submit_order(
            symbol=f'{ticker}',
            side='buy',
            type='market',
            qty=f'{amount}',
            # TODO: Do some calculation to get max amount under some percentage of current cash
            time_in_force='day'
        )

market_order_from_list(orderList)

api.list_orders()
api.cancel_all_orders()
